In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv('processed_dataset.csv')

In [3]:
df.shape

(75568, 54)

In [4]:
df = df.loc[(df['homeStatus'] <= 2)]
df = df.loc[(df['price'] > 100)]

In [5]:
np.random.seed(0)
y = df['price']
X = df.drop('price', axis=1)

# split into train and test
test_ratio = 0.2
train_indices = np.sort(np.random.choice(X.shape[0], int((1-test_ratio)*X.shape[0]), replace=False))
test_indices = np.setdiff1d(np.arange(X.shape[0]), train_indices)

train_indices = train_indices[:int(len(train_indices))]
test_indices = test_indices[:int(len(test_indices))]

X_train = X.iloc[train_indices]
y_train = y.iloc[train_indices]
X_test = X.iloc[test_indices]
y_test = y.iloc[test_indices]

In [6]:
imputer = KNNImputer(n_neighbors=5)
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
df.to_csv('imputed_scaled_dataset_0.csv', index=True)

In [ ]:
if 1:
    tree_regressor = DecisionTreeRegressor(min_samples_leaf=15)
else:
    tree_regressor = RandomForestRegressor(min_samples_leaf=15)

tree_regressor = tree_regressor.fit(X_train, y_train)
y_predictions = tree_regressor.predict(X_test)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X_train, y_train)
print(clf.feature_importances_  )
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X_test)

In [ ]:
np.sqrt(np.mean(np.abs(y_predictions-y_test)**2))